In [24]:
from praw import Reddit
from praw.models import Subreddit, Submission
import os
from dotenv import load_dotenv
from datetime import datetime, date, timedelta
import praw
import pandas as pd

In [25]:
# Initialize the Reddit client
reddit = praw.Reddit(
    client_id="4chqKwmOdorSuyav7KTIMg",      # Replace with your client ID
    client_secret="hI_HHldkP4pvfAsJ4xH9pWAkhGIn0g",  # Replace with your client secret
    user_agent="Sentiment-analysis",      
)

# Fetch posts from a subreddit with a keyword filter
def fetch_posts_about_keyword(subreddit_name, keywords, limit=100):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []
    
    for post in subreddit.hot(limit=limit):  # You can use new, top, etc.
        if any(keyword.lower() in (post.title + post.selftext).lower() for keyword in keywords):
            posts.append({
                'title': post.title,
                'score': post.score,
                'id': post.id,
                'url': post.url,
                'num_comments': post.num_comments,
                'created': post.created,
                'body': post.selftext
            })
    
    return posts

# Example usage
subreddit_name = "stocks"  # Replace with your subreddit of interest
keyword = ["Uber", "youtube", "meta", "apple","nvidia"," microsoft", "amazon",  "Saudi Arabian Oil Co", "intel", "tesla" , "MARA Holdings"]
data = fetch_posts_about_keyword(subreddit_name, keyword, limit=100)

# Print the fetched data
for post in data:
    print(post)

df = pd.DataFrame(data)
df.to_csv("Reddit.csv",index=False)

{'title': 'Do you think Rivian would make it to $100 a share?', 'score': 0, 'id': '1h3465k', 'url': 'https://www.reddit.com/r/stocks/comments/1h3465k/do_you_think_rivian_would_make_it_to_100_a_share/', 'num_comments': 118, 'created': 1732939670.0, 'body': 'With talks of Rivian saving VW, California keeping tax credit for Rivian and not Tesla, EV drivers turning in their Tesla’s for Rivian’s, the $35k Rivian coming out next year etc etc… Do you think it’s wise to buy a few hundred shares? Currently Rivian is at $12 a share, which is low. '}
{'title': 'Reddit overtakes X in popularity of social media platforms in UK $RDDT', 'score': 978, 'id': '1h1x2aj', 'url': 'https://www.reddit.com/r/stocks/comments/1h1x2aj/reddit_overtakes_x_in_popularity_of_social_media/', 'num_comments': 158, 'created': 1732803922.0, 'body': '[https://www.theguardian.com/technology/2024/nov/28/reddit-overtakes-x-in-popularity-of-social-media-platforms-in-uk](https://www.theguardian.com/technology/2024/nov/28/reddit